In [2]:
library("tidyverse")
library("data.table")
library("cowplot")
library("ggh4x")
library("ggrastr")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




In [3]:
test_hmm_colors <- c(
    "1" = "#440154",
    "2" = "#481b6d",
    "3" = "#46327e",
    "4" = "#3f4788",
    "5" = "#365c8d",
    "6" = "#2e6e8e",
    "7" = "#277f8e",
    "8" = "#21918c",
    "9" = "#1fa187",
    "10" = "#2db27d",
    "11" = "#4ac16d",
    "12" = "#73d056",
    "13" = "#a0da39",
    "14" = "#d0e11c",
    "15" = "#fde725",
    "ns" = "gray"
)

In [4]:
test_sig <- list(
    "of" = c("1", "2", "3", "4", "5", "6", "7", "8", "9", "13", "14"),
    "no" = c("1", "2", "3", "4", "5", "6", "7", "9")
)

In [5]:
strain_labels <- c("iCab" = "icab", "HdrR" = "hdr", "Ho5" = "ho5", "Kaga" =  "kaga", "HNI" = "hni")

In [ ]:
df <- fread("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states15_hmm.csv.gz")
df[, mean_dist := log10(mean(distance)), by = hmm_state]
tmp <- df[, .(hmm_state, mean_dist)] |> distinct() |> as.data.table()
tmp[, hmm_state_recoded := rank(mean_dist)]
df <- merge(df, tmp, by = c("hmm_state", "mean_dist"))
df[, hmm_state := hmm_state_recoded]
df[, hmm_state_recoded := NULL]
df <- df |>
    separate(
        id,
        into = c("date", "time_string", "ref_strain", "test_strain", "tank_side", "assay", "quadrant", "fish_type"),
        sep = "_",
        remove = FALSE
    ) |>
    as.data.table()
df[, test_strain := fct_recode(test_strain, !!!strain_labels)]
df[, test_strain := fct_relevel(test_strain, function(x){names(strain_labels)})]
head(df)

In [ ]:
f_list <- list.files("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/trajectories_csv/", full.names = TRUE)
df_positions <- lapply(f_list, function(f){fread(f)[, id := basename(f) |> str_remove("_traj_with_identities.csv.gz")]}) |> rbindlist(use.names = TRUE)
message("Read completed")
df_positions[, is_ref_ref := grepl("icab_icab", id)]
df_positions[, frame_n := 1:.N, by = id]
df_positions <- melt(
    df_positions[, .(id, frame_n, ref_x, ref_y, test_x, test_y)],
    id.vars = c("id", "frame_n")
)[
    , .(
        id,
        frame_n,
        axis = str_remove(variable, "^.*_"),
        fish_type = str_remove(variable, "_.*$"),
        value
    )
]
message("Melt completed")
df_positions <- dcast(df_positions, id + frame_n + fish_type ~ axis, measure.var = "value")
message("Dcast completed")
df_positions <- df_positions |>
    separate(
        id,
        into = c("date", "time_string", "ref_strain", "test_strain", "tank_side", "assay", "quadrant"),
        sep = "_",
        remove = FALSE
    ) |>
    as.data.table()
message("Separate completed")
df_positions[, test_strain := fct_recode(test_strain, !!!strain_labels)]
df_positions[, test_strain := fct_relevel(test_strain, function(x){names(strain_labels)})]
head(df_positions)

# Panel A

In [ ]:
plt_df <- df[
    assay == "of" &
    (fish_type == "test" | test_strain == "iCab") # we can use both icab as test fish in icab/icab pairs
]

In [ ]:
plt_df_positions <- df_positions[
    assay == "of" &
    (fish_type == "test" | test_strain == "iCab") # we can use both icab as test fish in icab/icab pairs
]

In [ ]:
plt_df[, hmm_state_sig := ifelse(as.character(hmm_state) %in% test_sig[["of"]], as.character(hmm_state), "ns")]
plt_df[
    , hmm_state_sig := factor(
        hmm_state_sig,
        levels = c(unique(hmm_state) |> sort(), "ns")
    )
]

In [ ]:
pA <- ggplot(plt_df, aes(x = time_s, y = id, fill = as.factor(hmm_state))) +
    geom_tile_rast() +
    scale_fill_manual(values = test_hmm_colors) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        legend.position = "none",
        legend.key.width = unit(0.5, "cm"),
        legend.key.height = unit(0.2, "cm"),
        axis.text.x = element_text(size = 9),
        axis.title.y = element_text(margin = margin(r = 10)),
        legend.title = element_text(size = 13, margin = margin(r = 20))
    ) +
    guides(fill = guide_legend(nrow = 3)) +
    labs(y = "Individual fish", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 8)
pA

# Panel B-alt

In [ ]:
pB_alt <- ggplot(plt_df, aes(x = time_s, fill = as.factor(hmm_state))) +
    geom_density(position = "fill") +
    scale_fill_manual(values = test_hmm_colors) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    scale_y_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        legend.position = "left",
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
    ) +
    guides(fill = guide_legend(ncol = 1, override.aes = list(color = NA))) +
    labs(y = "Density", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8)
pB_alt

# Panel B

In [ ]:
pB <- ggplot(plt_df, aes(x = time_s, fill = hmm_state_sig)) +
    geom_density(position = "fill", show.legend = TRUE) +
    scale_fill_manual(
        values = test_hmm_colors,
        drop = FALSE,
        labels = function(x){
            ifelse(x == "ns", "Not sig.", x)
        }
    ) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    scale_y_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        legend.position = "left",
        strip.background = element_blank(),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
    ) +
    guides(fill = guide_legend(ncol = 1, override.aes = list(color = NA))) +
    labs(y = "Density", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8)
pB

# Panel C

In [ ]:
pC <- ggplot(plt_df_positions, aes(x = x, y = y)) +
    stat_density_2d(aes(fill = after_stat(density)), geom = "raster", contour = FALSE) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) +
    scale_y_reverse(expand = expansion(mult = c(0, 0))) + # y is up to down
    scale_fill_distiller(palette = "RdBu") +
    facet_wrap(
        ~ test_strain,
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        legend.position = "right",
        legend.text = element_text(size = 9),
        axis.title.x = element_text(margin = margin(t = 4)),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
        legend.title = element_text(size = 13, margin = margin(b = 20))
    ) +
    labs(y = "Position y (px)", x = "Position x (px)", fill = "Density")

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 8)
pC

# ABC panel

In [ ]:
pABC <- plot_grid(
    pA, pB_alt, pC,
    nrow = 1,
    rel_widths = c(5, 6, 3),
    align = "h",
    axis = "tb",
    labels = c("A", "B", "C"),
    label_x = c(0, 0.2, 0)
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
pABC

In [ ]:
title_ABC <- ggdraw() + 
  draw_label(
    "Open field assay",
    fontface = 'bold',
    x = 0.5,
    hjust = 0.5,
    size = 18
  ) +
  theme(
    # add margin on the left of the drawing canvas,
    # so title is aligned with left edge of first plot
    plot.margin = margin(0, 0, 0, 7)
  )

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 0.5)
title_ABC

In [ ]:
pABC_title <- plot_grid(
    title_ABC,
    pABC,
    rel_heights = c(0.5, 8.5),
    ncol = 1
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
pABC_title

# Panel D

In [ ]:
plt_df <- df[
    assay == "no" &
    (fish_type == "test" | test_strain == "iCab") # we can use both icab as test fish in icab/icab pairs
]

In [ ]:
plt_df_positions <- df_positions[
    assay == "no" &
    (fish_type == "test" | test_strain == "iCab") # we can use both icab as test fish in icab/icab pairs
]

In [ ]:
plt_df[, hmm_state_sig := ifelse(as.character(hmm_state) %in% test_sig[["no"]], as.character(hmm_state), "ns")]
plt_df[
    , hmm_state_sig := factor(
        hmm_state_sig,
        levels = c(unique(hmm_state) |> sort(), "ns")
    )
]

In [ ]:
pD <- ggplot(plt_df, aes(x = time_s, y = id, fill = as.factor(hmm_state))) +
    geom_tile_rast() +
    scale_fill_manual(values = test_hmm_colors) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        legend.position = "none",
        legend.key.width = unit(0.5, "cm"),
        legend.key.height = unit(0.2, "cm"),
        axis.text.x = element_text(size = 9),
        axis.title.y = element_text(margin = margin(r = 10)),
        legend.title = element_text(size = 13, margin = margin(r = 20))
    ) +
    guides(fill = guide_legend(nrow = 3)) +
    labs(y = "Individual fish", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 8)
pD

# Panel E-alt

In [ ]:
pE_alt <- ggplot(plt_df, aes(x = time_s, fill = as.factor(hmm_state))) +
    geom_density(position = "fill") +
    scale_fill_manual(values = test_hmm_colors) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    scale_y_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        legend.position = "left",
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
    ) +
    guides(fill = guide_legend(ncol = 1, override.aes = list(color = NA))) +
    labs(y = "Density", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8)
pE_alt

# Panel E

In [ ]:
pE <- ggplot(plt_df, aes(x = time_s, fill = hmm_state_sig)) +
    geom_density(position = "fill", show.legend = TRUE) +
    scale_fill_manual(
        values = test_hmm_colors,
        drop = FALSE,
        labels = function(x){
            ifelse(x == "ns", "Not sig.", x)
        }
    ) + 
    scale_x_continuous(expand = expansion(mult = c(0, 0))) + 
    scale_y_continuous(expand = expansion(mult = c(0, 0))) + 
    facet_wrap(
        ~ test_strain,
        scales = "free_y",
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        legend.position = "left",
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
    ) +
    guides(fill = guide_legend(ncol = 1, override.aes = list(color = NA))) +
    labs(y = "Density", x = "Time (s)", fill = "HMM state")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 8)
pE

# Panel F

In [ ]:
pF <- ggplot(plt_df_positions, aes(x = x, y = y)) +
    stat_density_2d(aes(fill = after_stat(density)), geom = "raster", contour = FALSE) +
    scale_x_continuous(expand = expansion(mult = c(0, 0))) +
    scale_y_reverse(expand = expansion(mult = c(0, 0))) + # y is up to down
    scale_fill_distiller(palette = "RdBu") +
    facet_wrap(
        ~ test_strain,
        ncol = 1
    ) +
    theme_cowplot(15) +
    theme(
        axis.line = element_blank(),
        strip.background = element_blank(),
        panel.background = element_rect(fill = "gray"),
        legend.position = "right",
        legend.text = element_text(size = 9),
        axis.title.x = element_text(margin = margin(t = 4)),
        axis.title.y = element_text(margin = margin(r = 10)),
        axis.text = element_text(size = 9),
        legend.title = element_text(size = 13, margin = margin(b = 20))
    ) +
    labs(y = "Position y (px)", x = "Position x (px)", fill = "Density")

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 8)
pF

# DEF panel

In [ ]:
pDEF <- plot_grid(
    pD, pE_alt, pF,
    nrow = 1,
    rel_widths = c(5, 6, 3),
    align = "h",
    axis = "tb",
    labels = c("D", "E", "F"),
    label_x = c(0, 0.2, 0)
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
pDEF

In [ ]:
title_DEF <- ggdraw() + 
  draw_label(
    "Novel object assay",
    fontface = 'bold',
    x = 0.5,
    hjust = 0.5,
    size = 18
  ) +
  theme(
    # add margin on the left of the drawing canvas,
    # so title is aligned with left edge of first plot
    plot.margin = margin(0, 0, 0, 7)
  )

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 0.5)
title_DEF

In [ ]:
pDEF_title <- plot_grid(
    title_DEF,
    pDEF,
    rel_heights = c(0.5, 8.5),
    ncol = 1
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
pDEF_title

# Final

In [ ]:
p <- plot_grid(pABC_title, pDEF_title, ncol = 1)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 18)
p

In [ ]:
ggsave("fig5.pdf", p, width = 14, height = 18)